In [1]:
import pandas as pd
import numpy as np
import random
import sqlite3

Question 1.2

In [2]:
# Return the column names so that they don't have any space or '/'
def clean_col_names(col_list):
    
    ''' This is a function to clean the column names by:
    - Remove the spaces of the columns names
    - Remove the characters after '/'
    - Return col_list as a list
    '''
        
    col_list = list(col_list)
    for i in range(len(col_list)):

        col_list[i] = col_list[i].replace(' ', '')

        if "/" in col_list[i]:
            col_list[i] = col_list[i].split('/')[0]
    return col_list



# Convert numberic cols to float 
numeric_cols = ['Grade', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']
def numeric_columns(df):
    
    ''' This is a function converts the numeric columns from string to float type '''

    for i in numeric_cols:
        if i in df.columns:
            df[i] = pd.to_numeric(df[i], downcast = "float")
    return df



# Convert date columns to datetime
date_cols = ['Startedon', 'Completed']
def date_columns(df):
    
    ''' This is a function converts the date columns from object to datetime type '''
    
    for i in date_cols:
        if i in df.columns:
            df[i] = pd.to_datetime(df[i],errors = 'coerce')
    return df



# Replace null values with 0 
all_cols = ['Startedon', 'Completed','Grade', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 
            'Q10', 'Q11', 'Q12', 'Q13']
def remove_nulls(df):
    
    ''' This is a function to replace null values to zero by:
    - Selecting each of the columns of the dataframe
    - Fill na with 0
    - Replace '-' to zero
    '''
    
    for i in all_cols:
        if i in df.columns:
            df[i] = df[i].fillna(0)
            df[i] = df[i].replace({'-':'0'})
    return df
    

    
    
# Remove duplicates 
def remove_duplicates(df):
    
    ''' This is a function removes the duplicate grades by:
    - Sorting the values from the lowest to greatest grade
    - Keeps only the last grade which is the bigger
    '''

    df = df.sort_values('Grade').drop_duplicates(subset = ['researchid'], keep ='last')
    return df

  




#Function to clean the data 
def clean_data(temp):
    
    ''' This is a function to clean a dataframe by:
    - Clear the column names (clean_col_names function)
    - Replace null values with 0 (remove_nulls function)
    - Convert numeric columns to float (numeric_columns function)
    - Convert date columns to datetime (date_columns function)
    - Remove duplicated values (remove_duplicates function)
    - Drop specific columns
    '''

    
    #   Call function to rename col names
    df = temp.copy()
    df.columns = clean_col_names(df.columns) 

    
    #   Call function to replace null values with 0
    df = remove_nulls(df)
    
    
    #   Call function to convert numberic cols type from string to float 
    df = numeric_columns(df)
    
    #   Call the function that make the date columns type datetime
    df = date_columns(df)

    
    #   Call function that remove duplicates
    df = remove_duplicates(df)    
    
    
    #   Drop State and Timetaken columns
    df = df.drop(['State', 'Timetaken'], axis=1)
    return df


In [3]:
# original dataframes

# Read csv to pandas dataframes
dfFormative_Mock_Test = pd.read_csv('TestResult/Formative_Mock_Test.csv')
dfFormative_Test_1 = pd.read_csv('TestResult/Formative_Test_1.csv')
dfFormative_Test_2 = pd.read_csv('TestResult/Formative_Test_2.csv')
dfFormative_Test_3 = pd.read_csv('TestResult/Formative_Test_3.csv')
dfFormative_Test_4 = pd.read_csv('TestResult/Formative_Test_4.csv')
dfSumTest = pd.read_csv('TestResult/SumTest.csv')


In [4]:
# We add the raw data and clean them with the clean_data function

dfClean_Mock_Test = clean_data(dfFormative_Mock_Test)
dfCleanTest_1 = clean_data(dfFormative_Test_1)
dfCleanTest_2 = clean_data(dfFormative_Test_2)
dfCleanTest_3 = clean_data(dfFormative_Test_3)
dfCleanTest_4 = clean_data(dfFormative_Test_4)
dfClean_SumTest = clean_data(dfSumTest)

Question 1.3

In [5]:
# Find the normaliser factor
def find_normaliser_dict(col_list):
    
    ''' This is a function that find the normaliser factor
    - Uses the col_list as an input
    - Loops through the column list
    - If it find '/' in the loop, then it extract the value 
    - It splits the string
    - It keeps the second part (normaliser factor)
    - Returns a dictionary which maps the column with the normilisation factor
    '''
    
    col_list = list(col_list)
    values_dict = {}
    for i in range(len(col_list)):
        col_list[i] = col_list[i].replace(' ', '')
        if "/" in col_list[i]:
            values_dict[col_list[i].split('/')[0]] = int(col_list[i].split('/')[1])
    return values_dict


# Standardise all the columns which contain grades
def standardize_grade(df):
    
    ''' This is a function that standardise the grades to make them out of 100:
    - Uses find_normaliser_dict function to find the normaliser factor for each column (mapped in a dictionary)
    - Uses the clean_data function to clean the initial dataframes
    - Loops through all the numeric columns (defined outside the function) of the clean dataframes
    - Normalise the grade column
    '''
    
    norm_values = find_normaliser_dict(df.columns)
    clean_df = clean_data(df)
    for i in numeric_cols:
        if i in clean_df.columns:
            
            normaliser = 10000/norm_values[i]
            clean_df[i] = clean_df[i] * normaliser
    return clean_df



# Standardise the Grades with the use of standardise_grade function
dfFormattedClean_MockTest = standardize_grade(dfFormative_Mock_Test)
dfFormattedCleanTest_1 = standardize_grade(dfFormative_Test_1)
dfFormattedCleanTest_2 = standardize_grade(dfFormative_Test_2)
dfFormattedCleanTest_3 = standardize_grade(dfFormative_Test_3)
dfFormattedCleanTest_4 = standardize_grade(dfFormative_Test_4)
dfFormattedClean_SumTest = standardize_grade(dfSumTest)


Question 1.4

In [6]:

# Define the SQL connection
conn = sqlite3.connect ("ResultDatabase.db")

# Write dataframes to SQL database
dfFormattedClean_MockTest.to_sql("dfFormattedClean_MockTest",conn, if_exists = "replace", index = False)
dfFormattedCleanTest_1.to_sql("dfFormattedCleanTest_1", conn, if_exists = "replace", index = False)
dfFormattedCleanTest_2.to_sql("dfFormattedCleanTest_2", conn, if_exists = "replace", index = False)
dfFormattedCleanTest_3.to_sql("dfFormattedCleanTest_3", conn, if_exists = "replace", index = False)
dfFormattedCleanTest_4.to_sql("dfFormattedCleanTest_4", conn, if_exists = "replace", index = False)
dfFormattedClean_SumTest.to_sql("dfFormattedClean_SumTest", conn, if_exists = "replace", index = False)

# Close the connection
conn.close()

Question 1.5

In [7]:
# Generate 20 random indices from 0 - 150
random_indices = np.random.choice(150,20, replace = False)

# Select 3 random Question columns from column (Q1 -Q13)
random_Questions = random.sample(range(4,16),3)

# Find and choose the dataframe with the random IDs and Questions
Selectedrows = dfFormattedClean_SumTest.loc[random_indices,:].iloc[:,random_Questions]

# Create a new dataframe with the new random grades and renaming columns questions
dfrandom = pd.DataFrame(np.random.randint(0,100,(20,3)), index = random_indices)
dfrandom.columns = Selectedrows.columns.values 

# Updating the SumTest with the new random question grades
dfFormattedClean_SumTest.update(dfrandom)




# Find the normaliser and then change the Total Grades 

num_cols = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']

norm = find_normaliser_dict(dfSumTest)
dfedit = dfFormattedClean_SumTest.copy()

for i in num_cols:
     if i in dfedit.columns:
        normaliser = norm[i] / 10000
        dfedit[i] = dfedit[i] * normaliser
        dfedit['Grade'] =  dfedit.iloc[:,4:17].sum(axis =1)
        
dfedit.sort_index()



# Updating the column of Total Grades
dfGrades = dfedit.iloc[:,3]
dfFormattedClean_SumTest.update(dfGrades)



# Call SQL connection
conn = sqlite3.connect ("ResultDatabase.db")

# Write the dataframe to SQL database
dfFormattedClean_SumTest.to_sql("dfFormattedClean_SumTest", conn, if_exists = "replace", index = False)

# Close the connection
conn.close()